### Read Temperature with Arduino and VB.Net GUI

#### Overview:
    1. Prelude
    2. Setting up Arduino
    3. Setting up VB.Net
    4. Demonstration
    5. Codes reusability
    6. References

***

#### 1. Prelude

Using VB.Net, we can create a GUI application that can be converted into an executable file that can be run on windows. To do so, we must program both Arduino and VB.Net (see below). On Arduino, the program reads signals from DHT11 and sends to the computer temperature and humidity (see <b>section 2</b>). On VB.Net, temperature and humidity are read as a string, which is parsed by a comma and displayed separately on the GUI (see <b>section 3</b>). 

#### 2. Setting up Arduino [1]

#### 2.1. Parts and Circuit
*(a) Parts*  
> -DHT11 sensor (1)  
-5 kOhm Resistor (1)  
-Wires (3)  

*(b) Circuit*  
DHT11 has three connections:    

>    -Signal - connect to pin 2   
    -Voltage - connect to 5V  
    -Ground - connect to GND  

<p align="left"><b> Figure 1.1. Schematics </b></p>
<img src=Images/Fig_1-1_Schematics.png width=200 align="left">



*(c) The Program*

First, remember to install the SimpleDHT Library. To do so, download the zip file from this link: https://www.arduinolibraries.info/libraries/simple-dht. Then, on Arduino software go to Sketch -> Include library -> Add .ZIP Library..., locate the file downloaded zip file, and click open. The library will be added.

To verify that the library has been installed, again go to Sketch -> Include Library -> Manage Libraries... A window will pop out. In the textbox at the top right hand, type simpleDHT. If the library was successfully installed, you will see the INSTALLED message. At this point, you are ready to upload the Arduino program. Start a new script. Copy and paste the code below [1] and save the program as DHT11. Compile the program and then upload. Remember to set the correct COM port by going to Tools -> Port. The code below is fully copied from the CD that comes with ELEGOO UNO Project Super Starter Kit (https://www.elegoo.com/product/elegoo-uno-project-super-starter-kit/). 

In [ ]:
//www.elegoo.com
//2016.06.13

#include <SimpleDHT.h>

// for DHT11, 
//      VCC: 5V or 3V
//      GND: GND
//      DATA: 2
int pinDHT11 = 2;
SimpleDHT11 dht11;

void setup() {
  Serial.begin(9600);
}

void loop() {
  // start working...
  Serial.println("=================================");
  Serial.println("Sample DHT11...");
  
  // read with raw sample data.
  byte temperature = 0;
  byte humidity = 0;
  byte data[40] = {0};
  if (dht11.read(pinDHT11, &temperature, &humidity, data)) {
    Serial.print("Read DHT11 failed");
    return;
  }
  
  Serial.print("Sample RAW Bits: ");
  for (int i = 0; i < 40; i++) {
    Serial.print((int)data[i]);
    if (i > 0 && ((i + 1) % 4) == 0) {
      Serial.print(' ');
    }
  }
  Serial.println("");
  
  Serial.print("Sample OK: ");
  Serial.print((int)temperature); Serial.print(" *C, ");
  Serial.print((int)humidity); Serial.println(" %");
  
  // DHT11 sampling rate is 1HZ.
  delay(1000);
}

*(d) Testing the program*  
To view the temperature and humidity reading, go to tools -> Serial Monitor to bring out the Serial monitor window (<b>Figure 2.1 </b>).  
<br>

<b><center>Figure 2.1. Arduino serial monitor window. </center></b>

(a)             |  (b)
:-------------------------:|:-------------------------:
<img src=Images/Fig_2-1a_Serial_monitor_1.png width=300>  |  <img src=Images/Fig_2-1b_Serial_monitor_2.png width=300>


*(e) Modifying the program*  

For the purpose of monitoring a livestream data on VB.Net GUI, let's modify the code as follows:

In [ ]:
//Code is taken from: www.elegoo.com
//2016.06.13

#include <SimpleDHT.h>

// for DHT11, 
//      VCC: 5V or 3V
//      GND: GND
//      DATA: 2
int pinDHT11 = 2;
SimpleDHT11 dht11;

void setup() {
  Serial.begin(9600);
}

void loop() {

while (Serial.available() == 0);

int val = Serial.read() - '0';

if (val == 1){
  //If a signal of val==1 is received from VB.Net, read data from DHT11
  // start working... 
  // read with raw sample data.
  byte temperature = 0;
  byte humidity = 0;
  byte data[40] = {0};
  
  if (dht11.read(pinDHT11, &temperature, &humidity, data)) {
    //Serial.print("Read DHT11 failed");
    return;}
  
  //Separate the two values by a comma so that it can later be parsed
  Serial.print((int)temperature); Serial.print(" *C, ");
  Serial.print((int)humidity); Serial.println(" % ");
  
  // DHT11 sampling rate is 1HZ.
  delay(1000);}
 else if (val==0){
  //do nothing if signal of val==0 is received fro VB.Net
  }
}

#### 3. Setting up VB.Net 
#### 3.1. The VB.Net GUI
Below shows the form (<b>Figure 3.1</b>) that is associated with the code below. VB.Net is prepared using Visual Studio 2017.

<br>
<p align="left"><b> Figure 3.1.</b> VB.Net form.</p>

<img src=Images/Fig_3-1_VB-Net_GUI.png width=300 align="left">

####  3.2. Programming the VB.Net [2]

In [ ]:
Imports System.IO.Ports

Public Class Form1
    'Code is adapted from: http://www.martyncurrey.com/arduino-and-visual-basic-part-1-receiving-data-from-the-arduino/

    Dim WithEvents SerialPort As New IO.Ports.SerialPort
    Dim selPort As String

    Dim comPORT As String
    Dim receivedData As String = ""

    Private Sub Form1_Load(sender As Object, e As EventArgs) Handles MyBase.Load
        SPSetup()

        Timer1.Enabled = False

        comPORT = "" 'initialize comPORT as null
        For Each sp As String In My.Computer.Ports.SerialPortNames
            cmbComPort.Items.Add(sp)
        Next
        Try
            'Use the first entry as the default value
            cmbComPort.Text = cmbComPort.Items(0)
        Catch ex As Exception
            MsgBox("No ports are detected. Please check connections.", MsgBoxStyle.Exclamation, "Warning")
        End Try

    End Sub

    Public Sub SPSetup()    'Serial Port Setup
        On Error Resume Next
        If SerialPort.IsOpen Then
            SerialPort.Close()
        End If

        'Set communication parameters
        SerialPort.PortName = selPort  ' "COM3"
        SerialPort.BaudRate = 9600
        SerialPort.DataBits = 8
        SerialPort.StopBits = IO.Ports.StopBits.One
        SerialPort.Handshake = IO.Ports.Handshake.None
        SerialPort.Parity = IO.Ports.Parity.None

        SerialPort.Open() 'connect to Serial Port

    End Sub
    Private Sub btnExit_Click(sender As Object, e As EventArgs) Handles btnExit.Click

        Me.Close() 'close form

    End Sub
    Private Sub cmbComPort_SelectedIndexChanged(sender As Object, e As EventArgs) Handles cmbComPort.SelectedIndexChanged

        'set comPort variable to user selected value
        If (cmbComPort.SelectedItem <> "") Then
            comPORT = cmbComPort.SelectedItem
        End If

    End Sub
    Private Sub connect_BTN_Click(sender As Object, e As EventArgs) Handles connect_BTN.Click

        'If communication is off, start it and begin communication
        If (connect_BTN.Text = "Connect") Then
            If (comPORT <> "") Then
                SerialPort1.Close()
                SerialPort1.PortName = comPORT
                SerialPort1.BaudRate = 9600
                SerialPort1.DataBits = 8
                SerialPort1.Parity = Parity.None
                SerialPort1.StopBits = StopBits.One
                SerialPort1.Handshake = Handshake.None
                SerialPort1.Encoding = System.Text.Encoding.Default
                SerialPort1.ReadTimeout = 10000

                SerialPort1.Open()

                connect_BTN.Text = "Disconnect"
                Timer1.Enabled = True
                lblTimer.Text = "Timer: ON"

            Else 'If not connected, provide a warning
                MsgBox("Select a COM port first")
            End If
            'Close port and stop timer

        Else
            'Send command to Arduino to activate DHT11
            SerialPort1.Write("0")

            'Close the serial port
            SerialPort1.Close()

            connect_BTN.Text = "Connect"
            Timer1.Enabled = False
            lblTimer.Text = "Timer: OFF"
        End If

    End Sub
    Private Sub Timer1_Tick(sender As Object, e As EventArgs) Handles Timer1.Tick

        'As long as timer is on, send command to Arduino to activate DHT11
        SerialPort1.Write("1")

        'Obtain data via serial communication and update information in textbox
        receivedData = ReceiveSerialData()

        'try parsing incoming data
        Dim temp As New ArrayList() 're-initialize array each time
        Try
            For Each s As String In receivedData.Split(",")
                temp.Add(s) 'add to array
            Next
            lblTemperature.Text = temp(0)
            lblHumidity.Text = temp(1)
        Catch ex As Exception
            'If text is not parseable, show "-"
            lblTemperature.Text = "-"
            lblHumidity.Text = "-"
        End Try

    End Sub
    Function ReceiveSerialData() As String

        'read data via serial communication
        Dim Incoming As String
        Try
            'check for incoming signals
            Incoming = SerialPort1.ReadExisting()

            If Incoming Is Nothing Then
                Return "nothing" & vbCrLf
            Else
                Return Incoming
            End If

        Catch ex As TimeoutException
            'Return "Error: Serial Port read timed out."
            Return ""
        End Try

    End Function

End Class


#### 4. Demonstration

<b> Figure 4.1 </b> below shows the GUI in the off and on states.

<b><center>Figure 4.1. VB.Net GUI. </center></b>

(a) Off-state            |  (b) On-state
:-------------------------:|:-------------------------:
<img src=Images/Fig_4-1a_VB-Net_GUI_off.png width=300>  |  <img src=Images/Fig_4-1b_VB-Net_GUI_on.png width=300>

#### 5. Codes reusability

- The VB.Net GUI is reusable. It can be modified only slightly to read any incoming serial data.
- The Arduino code is reusable. The same script can be used to connect with other types of sensors. I recommend experimenting with the 45 in 1 sensor module for your sensing needs (e.g., https://www.amazon.com/VKmaker-Sensors-Modules-Starter-Arduino/dp/B01CS6UMKQ). 


#### 6. References

[1] "How to Set Up the DHT11 Humidity Sensor on an Arduino", Posted by Circuit Basics, Retrieved on 08/16/18, URL: 
http://www.circuitbasics.com/how-to-set-up-the-dht11-humidity-sensor-on-an-arduino/  
[2] "Arduino and Visual Basic Part 1: Receiving Data From the Arduino", Retrieved on 08/18/18, Martyn Currey, URL: http://www.martyncurrey.com/arduino-and-visual-basic-part-1-receiving-data-from-the-arduino/     